# Скоринговая модель прогнозирования вероятности дефолта заемщика банка



## Import

In [1]:
from pandas import Series
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, RobustScaler, PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, mean_squared_error
from sklearn.metrics import auc, roc_auc_score, roc_curve

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

Фиксируем random_seed

In [2]:
random_seed=42

#### Функции

In [3]:
def model_metrics():
    print('precision_score:', round(precision_score(y_test,y_pred), 3))
    print('recall_score:', round(recall_score(y_test,y_pred), 3))
    print('f1_score:', round(f1_score(y_test,y_pred), 3))
    cm = confusion_matrix(y_test, y_pred)
    print('confusion_matrix:\n', cm)
    print('MSE: {}'.format(np.round(mean_squared_error(y_test, y_pred), 4)))

## Data

In [4]:
DATA_DIR = '/kaggle/input/sf-dst-scoring/'
train = pd.read_csv(DATA_DIR+'/train.csv')
test = pd.read_csv(DATA_DIR+'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

train = pd.read_csv('train_scoring.csv')
test = pd.read_csv('test_scoring.csv')
sample_submission = pd.read_csv('sample_submission_scoring.csv')

#### Описание признаков

Для корректной обработки признаков объединяем обучающие данные и тестовые в один датасет с соответствующими метками: 

In [5]:
train['sample'] = 1
test['sample'] = 0
# в данных для теста нет значений целевой, так что добавим этот признак с явно отличным значением
test['default'] = -1 
data = pd.concat([train, test], ignore_index=True)

Посмотрим на признаки более детально:

In [6]:
data_agg = data.agg({'nunique', lambda s: s.unique()[:10]})\
    .append(pd.Series(data.isnull().sum(), name='null'))\
    .append(pd.Series(data.dtypes, name='dtype'))\
    .transpose()
data_agg

#### Обработка пропусков

Единственный признак с пропусками данных - образование. Поскольку это категориальный признак, пропуски заполним наиболее часто встречающимся значением.
Посмотрим, какое это значение: 
    

In [7]:
sns.countplot(x='education', data = data, palette = 'Set1')

In [8]:
data['education'] = data['education'].fillna('SCH')

### Обработка дат    

Сама по себе дата малоинформативна, и ценность будет представлять оценка срока давности подачи заявки на рассмотрение с учетом самой ранней даты в датасете. 
    

In [9]:
data['app_date'] = data['app_date'].apply(lambda x: (pd.Timestamp.today() - pd.to_datetime(x)).days)
data['app_date'] = data['app_date'] - min(data['app_date'])

Разделим признаки по типам:

In [10]:
num_cols = ['app_date', 'age', 'decline_app_cnt', 'score_bki', 'bki_request_cnt', 'income']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']
cat_cols = ['education', 'region_rating', 'home_address', 'work_address', 'sna', 'first_time']

### Обработка бинарных признаков 

Посмотрим соотношение бинарных признаков к целевой переменной

In [11]:
def features_relation(data, columns):
    sns.set_style('darkgrid')
    
    fig = plt.figure(figsize=(35,10))
       
    for i, value in enumerate(columns):
        plt.subplot(1,len(columns),i+1)
        sns.barplot(x=value, y="default", data=data[[value, 'default']])
        fig.tight_layout(pad=1.0)
        sns.set_context('talk',font_scale=1.1)
    
    fig.subplots_adjust(top=0.8)

features_relation(train, bin_cols)

Стало видно, что: 

    - мужчины чуть чаще допускают дефолт, чем женщины, 
    - владельцы машин - реже, чем заемщики без машин, 
    - владльцы иномарок - реже, чем владельцы других авто, 
    - заемщики с хорошей работой реже не выплачивают кредит, чем остальные, 
    - владельцы загранпаспорта - реже не имеющих загранпаспорта. 

адаптируем бинарные признаки для модели:

In [12]:
bin_col_labels = {}
label_encoder = LabelEncoder()
for i in bin_cols:
    data[i] = label_encoder.fit_transform(data[i])
    bin_col_labels[i] = dict(enumerate(label_encoder.classes_))
bin_col_labels

### Обработка категориальных признаков 

In [13]:
cat_col_labels = {}
label_encoder = LabelEncoder()
for i in cat_cols:
    data[i] = label_encoder.fit_transform(data[i])
    cat_col_labels[i] = dict(enumerate(label_encoder.classes_))
cat_col_labels

Посмотрим на взимосвязь категориальных признаков и целевой: 

In [14]:
features_relation(train, cat_cols)

Распределения показыают, что: 
    
    - заемщики с уровнем образования ACD допускают дефолт реже остальных категорий,
    - заемщики из регионов проживания с высоким рейтингом наиболее успешны в погашении кредитов,
    - заемщики с домашними адресами первой категории вылачивают кредиты лучше двух других категорий, аналогично с категориями рабочих адресов, 
    - наличие связей с клиентами банка 1 категории характеризует успешных заемщиков, 
    - категория давности информации о заемщике в базе обратнопропорциональна количеству дефолтов в категории.
    


### Значимость категориальных и бинарных признаков для модели 

In [15]:
imp_cat = Series(mutual_info_classif(data[bin_cols + cat_cols],
                                     data['default'], discrete_features =True),
                                     index = bin_cols + cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Интересно, что наличие связи заемщика с работниками банка имеет самую большую ценность для модели, и при этом наличие машины не так важно, как какая именно машина есть у заемщика. Значимость пола же вообще очень низка.

### Обработка числовых признаков

Посмотрим на распределение числовых признаков: 

In [16]:
for i in num_cols:
    plt.figure()
    sns.distplot(data[i][data[i] > 0].dropna(), kde = False, rug=False)
    plt.title(i)
    plt.show()

Также посмотрим на распределение числовых переменных по отношению к целевой: 

In [17]:
for i in num_cols:
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=i, 
                data=data[data['default']==1],
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot для дефолт-заемщиков, ' + i)
    plt.show()
    fig, ax = plt.subplots(figsize = (14, 4))
    sns.boxplot(x=i, 
                data=data[data['default']==0],
               ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot для успешных заемщиков, ' + i)
    plt.show()

Boxplot-ы  показывают, что в среднем, успешные заемщики:
    
    -по возрасту ближе к 40 годам, чем дефолт-заемщики, 
    -количество отказных заявок у них в среднем меньше, но есть и заметно отклоняющиеся показатели, 
    -скоринговый балл БКИ у успешных в среднем меньше, 
    -количество запросов кредитной историии в БКИ у успешных заемщиков меньше, чем у дефолт-заемщиков;
    -показатели дохода у успешных заемщиков смещены в большую сторону.

Числовые признаки показывают смещенное распределение, с длинным правым хвостом, перед обучением модели попробуем нормализовать их для лучшего обучения. 

#### Выбросы

Бокс плоты и описания признаков показали, что аномальных значений, выпадающих за границы здравого смысла нет. Отсекать дополнительные данные нерезонно. 

#### Корреляция числовых признаков с целевой

In [18]:
corr = train[num_cols + ['default']].corr()
f, ax = plt.subplots(figsize=(12,8))
sns.heatmap(corr, vmax=1, vmin=-1, annot=True)

Все признаки довольно слабо скоррелированы с признаком дефолта, так что оставим все.

#### Проверка на сбалансированность классов целевой переменной

In [19]:
sns.countplot(x ='default', data = train, palette='Set2')

In [20]:
print("Успешных заемщиков:", train.default.value_counts()[0],
             "Дефолт-заемщиков:", train.default.value_counts()[1])

Классы целевой представлены неравномерно, есть дисбаланс. 
Это накладывает ряд следующих ограничений: 
        
    1). Метрики оценки качества модели должны учитывать дисбаланс. Такими являются precision, recall, f1-score, roc-кривые. Accuracy  точно не подойдет.
    2). Для настройки модели полезно будет попробовать настройку гиперпараметров и наращивание представителей класса.

#### Значимость непрерывных переменных 

В основе метода оценки значимости переменных лежит однофакторный дисперсионный анализ (ANOVA).
В качестве меры значимости будем использовать значение f-статистики. Чем значение статистики выше, тем меньше вероятность того, что средние значения не отличаются, и тем важнее данный признак для нашей линейной модели.

In [21]:
temp_df = data[data['sample'] == 1]
imp_num = Series(f_classif(temp_df[num_cols], temp_df['default'])[0],
                 index=num_cols)
imp_num.sort_values(inplace=True)
imp_num.plot(kind='barh')

Наибольшим f-score обладает скоринговый балл БКИ - $score\_bki$ , и количество отказных кредитных заявок - $decline\_app\_cnt$; эти признаки демонстрируют высокую значимость.

In [22]:
corr_data = data.drop(['sample', 'client_id'], axis=1).corr()
f, ax = plt.subplots(figsize=(24,10))
sns.heatmap(corr_data, vmax=1, vmin=-1, annot=True)

У нас есть признаки, сильно линейно связанные - car  и car_type, и home_adress  и work_adress.
Преобразуем их.

In [23]:
data.drop(['car'], axis=1, inplace=True)


In [24]:
bin_cols.remove('car')

Попробуем объединить рабочий и домашний адреса методом декомпозиции.

In [25]:
# Выберем из датасета нужные колонки:
df = data[['work_address', 'home_address']].values

# Создадим Scaler instance:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df)

# Из двух столбцов сделаем один путем усечения ненужной информации.
pca = PCA(n_components=1)
pca.fit(scaled_data)
pca_data = pca.transform(scaled_data)
data['address'] = pca_data

# Уберем ненужные колонки:
data = data.drop(['home_address','work_address'],axis=1)

# Приведем в порядок списки:
cat_cols.remove('home_address')
cat_cols.remove('work_address')
cat_cols.append('address')

In [26]:
cat_cols

### Наивная модель 

##### Выделение обучающих данных и целевой

Для обучения будут использоваться данные из train части data

Разбиение 

In [27]:
data_train = data[data['sample'] == 1].drop(['sample'], axis=1)

x = data_train.drop(['default'], axis = 1)
y = data_train['default']

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=random_seed)

In [29]:
naive_model = LogisticRegression()
naive_model.fit(x_train, y_train)

In [30]:
y_pred = naive_model.predict(x_test)

In [31]:
probs = naive_model.predict_proba(x_test)
probs

In [32]:
probs = probs[:,1]

Построим ROC-кривую наглядно для наивной модели

In [33]:
fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

Наивная модель предсказывает немногим лучше случайного отгадывания.

Метрики и матрица ошибок наглядно:

In [34]:
model_metrics()

conf_mat = confusion_matrix(y_test, y_pred)
sns.set(font_scale=1.4) 
sns.heatmap(conf_mat, annot=True, annot_kws={"size": 16}, fmt='g')

Confusion matrix показывает, что модель допускает много ошибок типа False Negative, что на практике оборачивается для банка солидными финансовым потерями, полностью игнорирует отнесение к истинно не-дефолтным клиентам - следствие несбалансированности классов целевой переменной.   

## Feature engineering

Сгладим распределение числовых признаков логарифмированием. Экспериментально лучше всего работает логарифмирование дохода

In [35]:
num_cols_log = ['income']
for column in num_cols_log:
    data[column] = np.log(data[column] + 5)

Попробуем добавить полиномиальные признаки: 
    

In [36]:
pf = PolynomialFeatures(2, include_bias=False)
poly_data = pf.fit_transform(data[num_cols])[:, len(num_cols):]
poly_cols = pf.get_feature_names()[len(num_cols):]
poly_df = pd.DataFrame(poly_data, columns=poly_cols)
data = data.join(poly_df, how='left')

## Модель 

выполним разделение заново 

In [37]:
train = data.query('sample == 1').drop(['sample'], axis=1)
test = data.query('sample == 0').drop(['sample'], axis=1)

In [38]:
# Проверим на соответствие значений:
print('Размер тренировочного датасета: ', train.shape,
      'Размер тестового датасета: ', test.shape, sep='\n')

In [39]:
# Выводим категориальные переменные в 0 или 1.
X_cat = OneHotEncoder(sparse=False).fit_transform(train[cat_cols].values)

Сгладим распределения непрерывных признаков, уменьшив влияние сильно отстоящих значений.

In [51]:
num_cols

In [40]:
# Стандартизация числовых переменных:
X_num = StandardScaler().fit_transform(train[num_cols].values)

In [41]:
# Объединим стандартизованные числовые, полиномиальные, бинарные и закодированные категориальные переменные в одно 
# признаковое пространство,разделив при этом признаки и целевую переменную.
X = np.hstack([X_num, train[bin_cols].values, train[poly_cols].values, X_cat])
Y = train['default'].values
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=random_seed)

In [42]:
X_test.shape

In [43]:
model = LogisticRegression(solver='liblinear', class_weight='balanced', random_state=random_seed, max_iter = 1000, penalty='l2', C=1)

In [44]:
model.fit(X_train, y_train)

In [45]:
y_pred = model.predict(X_test)

In [55]:
y_pred.shape

In [46]:
probs = model.predict_proba(X_test)
probs = probs[:,1]

In [47]:

probs.shape


In [48]:
model_metrics()

conf_mat = confusion_matrix(y_test, y_pred)
sns.set(font_scale=1.4) 
sns.heatmap(conf_mat, annot=True, annot_kws={"size": 16}, fmt='g')

Очень хотелось бы снизить значение FP=4292, дефолтных клиентов, которых классифицировали как заслуживающих доверия. Возможно, для этого необходимо больше данных для выявления закономерностей.

In [49]:
fpr, tpr, threshold = roc_curve(y_test, probs)
roc_auc = roc_auc_score(y_test, probs)

plt.figure()
plt.plot([0, 1], label='Baseline', linestyle='--')
plt.plot(fpr, tpr, label = 'Regression')
plt.title('Logistic Regression ROC AUC = %0.3f' % roc_auc)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc = 'lower right')
plt.show()

### Подбор гиперпараметров: 

# Добавим типы регуляризации
penalty = ['l1', 'l2']

# Зададим ограничения для параметра регуляризации
C = np.logspace(0, 4, 10)

# Создадим гиперпараметры
hyperparameters = dict(C=C, penalty=penalty)

model_hp = LogisticRegression()
model_hp.fit(X_train, y_train)

# Создаем сетку поиска с использованием 5-кратной перекрестной проверки
clf = GridSearchCV(model_hp, hyperparameters, cv=5, verbose=0)

best_model = clf.fit(X_train, y_train)

# View best hyperparameters
print('Лучшее Penalty:', best_model.best_estimator_.get_params()['penalty'])
print('Лучшее C:', best_model.best_estimator_.get_params()['C'])

поскольку в юпитере эта ячейка отработала нормально, оптимальные параметры все же высчитаны:

Лучшее Penalty: l2
Лучшее C: 1

## Submission

In [56]:
X_cat_test = OneHotEncoder(sparse = False).fit_transform(test[cat_cols].values)
X_num_test = StandardScaler().fit_transform(test[num_cols].values)
X_test_test = np.hstack([X_num_test, test[bin_cols].values, test[poly_cols].values, X_cat_test])
y_probs = model.predict_proba(X_test_test)[:,1]

Применяем полученный результат на тестовых данных

In [58]:
test['default'] = y_probs

In [59]:
submission = test[['client_id','default']]
display(submission.sample(10))
display(submission.shape)

In [60]:
submission.to_csv('submission.csv', index=False)
submission